# Evaluation Demo

## Portfolio Analysis

In [4]:
%%bash
curl https://benchmark-database.de/getdatabase/meta --output meta.db
curl https://satcompetition.github.io/2023/downloads/sc2023-detailed-results.zip --output results.zip
unzip results.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed


100 25.3M  100 25.3M    0     0  10.4M      0  0:00:02  0:00:02 --:--:-- 10.4M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  165k  100  165k    0     0   713k      0 --:--:-- --:--:-- --:--:--  715k


Archive:  results.zip
  inflating: bench_meta.csv          
  inflating: results_cloud_detailed.csv  
  inflating: results_main_detailed.csv  
  inflating: results_nolimit_detailed.csv  
  inflating: results_parallel_detailed.csv  
  inflating: results_special_detailed.csv  


In [27]:
from gbd_core.api import GBD
import pandas as pd

# This script computes the group-wise scores for the main track of the 2023 SAT Competition.

with GBD(['results_main_detailed.csv']) as gbd:
    solvers = [ s for s in gbd.get_features() if s not in ["aresult", "vresult"] ]

with GBD(['meta.db', 'results_main_detailed.csv']) as gbd:
    # get solver runtimes and instance families for the main track of the 2023 SAT Competition
    df = gbd.query("track=main_2023", resolve=solvers + ["family"], collapse='min')
    # convert runtimes to numeric values
    df[solvers] = df[solvers].apply(pd.to_numeric)
    # penalize timeouts
    for solver in solvers:
        df.loc[df[solver] >= 5000, solver] = 10000
    # compute the vbs score
    df["vbs"] = df[solvers].min(axis=1)
    # group families with less than 5 instances into a single group
    misc = df.groupby("family").count().query("hash < 5").index.tolist() + ["empty", "unknown"]
    df.replace(misc, "miscellaneous", inplace=True)
    # compute the group-wise scores
    counts = df.groupby('family').count()["hash"].rename("count")
    groups = df.groupby('family').mean(numeric_only=True)
    tab = groups.merge(counts, on='family', how='left').reset_index()
    tab["diff"] = tab[solvers].max(axis=1) - tab[solvers].min(axis=1)
    tab.sort_values(by="diff", ascending=False, inplace=True)
    tab = tab[["family", "count"] + solvers + ["vbs"]]
    # add the "all" row
    all = df.mean(numeric_only=True).to_frame("score").transpose()
    all["family"] = "all"
    all["count"] = len(df.index)
    tab = pd.concat([tab, all], ignore_index=True)
    print(tab)
    #tables.group_wise_scores(tab, solvers + [ "vbs" ], ["family"], "{}/{}.tex".format(self.target_dir, name), bold_min_of=solvers, min_diff=0)


                          family  count        AMSAT_  BreakID_kissat_low_sh  \
0              interval-matching     20  10000.000000           10000.000000   
1            register-allocation     20   8962.413500            2016.385696   
2                   set-covering     20      8.596939             262.389965   
3                     or_randxor      5  10000.000000             103.927660   
4                brent-equations     19    684.381491            1133.501221   
5               hashtable-safety     20    368.622450           10000.000000   
6             cryptography-ascon     20    361.009195            2673.768060   
7             school-timetabling     19   1129.047268            2371.415084   
8                        satcoin     15   1601.136267           10000.000000   
9           mutilated-chessboard     12   6144.539667            5135.770558   
10                   grs-fp-comm     17   6321.562294            8258.639147   
11                  cryptography      7 

## Category-wise Ranking

In [ ]:
from gbd_core.api import GBD

## Category Prediction